In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [29]:
def activation_function(net_input,ch):
    if(ch==1):
        return 1/(1+np.exp(-net_input))
    elif(ch==2):
        return (1-np.exp(-net_input))/(1+np.exp(-net_input))  

def derivative(net_input,ch):
    if(ch==1):
        return net_input*(1-net_input)    
    elif(ch==2):
        return (1/2)*(1+net_input)*(1-net_input)  

def predict(inputs, input_weights, hidden_weights, input_biases, hidden_bias, ch):
    hidden_input = np.dot(inputs, input_weights) + input_biases
    hidden_output=[]
    for i in range(len(hidden_input)):
        hidden_output.append(activation_function(i, ch))
    output_input = np.dot(hidden_output, hidden_weights) + hidden_bias
    output = activation_function(output_input[0], ch)
    return round(output)

def train_mlp(train, hidden_units, lr, ch, target):
    input_units = len(train.columns) - 1
    output_units = 1

    input_weights = np.random.rand(input_units, hidden_units)
    hidden_weights = np.random.rand(hidden_units, output_units)
    input_biases = np.zeros(hidden_units)
    hidden_bias = 0

    converged = False
    epoch = 0
    while not converged and epoch <= 10:
        converged = True
        for i in range(len(train)):
            hidden_input = np.dot(train.iloc[i, :target].astype(float), input_weights) + input_biases
            hidden_output = [activation_function(h_input, ch) for h_input in hidden_input]
            output_input = np.dot(hidden_output, hidden_weights) + hidden_bias
            output = activation_function(output_input, ch)

            error = train.iloc[i, target] - output
            delta_output = error * derivative(output, ch)

            hidden_error = delta_output.dot(hidden_weights.T)
            delta_hidden = hidden_error * np.array([derivative(h_out, ch) for h_out in hidden_output])

            hidden_weights += lr * np.outer(hidden_output, delta_output)
            hidden_bias += lr * delta_output

            input_weights += lr * np.outer(train.iloc[i, :target].astype(float), delta_hidden)
            input_biases += lr * delta_hidden

            if error != 0:
                converged = False
        epoch += 1

    print('\nFinal input weights:\n', input_weights)
    print('Final hidden weights:\n', hidden_weights)
    print('Input biases:', input_biases)
    print('Hidden bias:', hidden_bias, "\n")
    return input_weights, hidden_weights, input_biases, hidden_bias


def train_test_split(data,split_ratio):
    train_size=int(len(data)*split_ratio)
    print("LEN",train_size)
    train=data[:train_size]
    test=data[train_size:]
    return train,test

In [11]:

df=pd.read_csv(r"D:\SEM-5\ML-LAB\ex-1\new_forest_full_dataset.csv")
le=LabelEncoder()
df.columns=df.columns.str.strip()
df['Classes']=df['Classes'].str.strip()

In [8]:
df.columns

Index(['day', 'month', 'year', 'Region', 'Temperature', 'RH', 'Ws', 'Rain',
       'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes'],
      dtype='object')

In [12]:
df

,day,month,year,Region,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,2012,Bejaia,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,no
1,2,6,2012,Bejaia,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,no
2,3,6,2012,Bejaia,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,no
3,4,6,2012,Bejaia,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,no
4,5,6,2012,Bejaia,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,26,9,2012,Sidi-Bel-Abbes,30,65,14,0.0,85.4,16.0,44.5,4.5,16.9,6.5,yes
240,27,9,2012,Sidi-Bel-Abbes,28,87,15,4.4,41.1,6.5,8.0,0.1,6.2,0.0,no
241,28,9,2012,Sidi-Bel-Abbes,27,87,29,0.5,45.9,3.5,7.9,0.4,3.4,0.2,no
242,29,9,2012,Sidi-Bel-Abbes,24,54,18,0.1,79.7,4.3,15.2,1.7,5.1,0.7,no


In [14]:
df['Classes']=le.fit_transform(df['Classes'])
df['Region']=le.fit_transform(df['Region'])

In [19]:
df=df.drop('year',axis=1)

In [20]:
df

,day,month,Region,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,0,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,0
1,2,6,0,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,0
2,3,6,0,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,0
3,4,6,0,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,0
4,5,6,0,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,26,9,1,30,65,14,0.0,85.4,16.0,44.5,4.5,16.9,6.5,1
240,27,9,1,28,87,15,4.4,41.1,6.5,8.0,0.1,6.2,0.0,0
241,28,9,1,27,87,29,0.5,45.9,3.5,7.9,0.4,3.4,0.2,0
242,29,9,1,24,54,18,0.1,79.7,4.3,15.2,1.7,5.1,0.7,0


In [26]:
np.asarray(df)

array([[ 1. ,  6. ,  0. , ...,  3.4,  0.5,  0. ],
       [ 2. ,  6. ,  0. , ...,  3.9,  0.4,  0. ],
       [ 3. ,  6. ,  0. , ...,  2.7,  0.1,  0. ],
       ...,
       [28. ,  9. ,  1. , ...,  3.4,  0.2,  0. ],
       [29. ,  9. ,  1. , ...,  5.1,  0.7,  0. ],
       [30. ,  9. ,  1. , ...,  4.8,  0.5,  0. ]])

In [31]:

ts=float(input("Enter the test size:"))
train_size=1-ts

train,test=train_test_split(df,train_size)
print("\nTraining Set:\n",train,"\n\nTesting Set:\n",test,"\n")

lr=float(input("Enter the learning rate:"))
hidden=int(input("Enter the number of hidden units:"))
ch=int(input('\n1.Binary Sigmoidal Function\n2.Bipolar Sigmoidal Function\n\nEnter your choice:'))

target=13

iweights,hweights,ibias,hbias=train_mlp(train,hidden,lr,ch,target)

prediction=[]
for i in range(len(test)):    
    pred=predict(test.iloc[i,:target],iweights,hweights,ibias,hbias,ch)
    prediction.append(pred)
print('Sample prediction:\n')
print('Act O/P:',test.iloc[1,target],'Pred O/P:',prediction[1])

test=test.iloc[:,target]

acc=accuracy_score(test,prediction)
print("\nAccuracy:\n",acc)


Enter the test size:0.25
LEN 183

Training Set:
      day  month  Region  Temperature  RH  Ws  Rain  FFMC   DMC    DC  ISI   
0      1      6       0           29  57  18   0.0  65.7   3.4   7.6  1.3  \
1      2      6       0           29  61  13   1.3  64.4   4.1   7.6  1.0   
2      3      6       0           26  82  22  13.1  47.1   2.5   7.1  0.3   
3      4      6       0           25  89  13   2.5  28.6   1.3   6.9  0.0   
4      5      6       0           27  77  16   0.0  64.8   3.0  14.2  1.2   
..   ...    ...     ...          ...  ..  ..   ...   ...   ...   ...  ...   
178   27      7       1           29  87  18   0.0  80.0  11.8  28.3  2.8   
179   28      7       1           33  57  16   0.0  87.5  15.7  37.6  6.7   
180   29      7       1           34  59  16   0.0  88.1  19.5  47.2  7.4   
181   30      7       1           36  56  16   0.0  88.9  23.8  57.1  8.2   
182   31      7       1           37  55  15   0.0  89.3  28.3  67.2  8.3   

      BUI   FWI  Classes  